<a href="https://colab.research.google.com/github/pratiksrm99/Abstractive_Text_Summerization/blob/main/Abstractive_TS.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install datasets
!pip install pywsd
import nltk
nltk.download('wordnet')
nltk.download('omw-1.4')
nltk.download('averaged_perceptron_tagger')
nltk.download('punkt')

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [2]:
from datasets import load_dataset
from nltk.corpus import wordnet as wn
from nltk.tokenize import word_tokenize
from nltk import FreqDist
from pywsd.lesk import adapted_lesk, simple_lesk, cosine_lesk
import pandas as pd
import numpy as np
import spacy
import re

Warming up PyWSD (takes ~10 secs)... took 19.55627679824829 secs.
/usr/local/lib/python3.8/dist-packages/torch/cuda/__init__.py:497: UserWarning: Can't initialize NVML
  warnings.warn("Can't initialize NVML")


In [3]:
dataset = load_dataset('gigaword')

  0%|          | 0/3 [00:00<?, ?it/s]

In [4]:
df = pd.DataFrame(columns= ["Document", "Summary"])
df["Document"] = dataset.get("test")["document"]
df["Summary"] = dataset.get("test")["summary"]

In [5]:
# df["Split_text"] = df["Document"].apply(str.split)
# df["Article_length"] = df["Split_text"].apply(len)

In [6]:
df.head()

,Document,Summary
0,japan 's nec corp. and UNK computer corp. of t...,nec UNK in computer sales tie-up
1,the sri lankan government on wednesday announc...,sri lanka closes schools as war escalates
2,police arrested five anti-nuclear protesters t...,protesters target french research ship
3,factory orders for manufactured goods rose #.#...,us september factory orders up #.# percent
4,the bank of japan appealed to financial market...,bank of UNK UNK for calm in financial markets


In [7]:
sent = "he is sitting on the bank of the river watching a speedboat."
nlp = spacy.load("en_core_web_sm")

def one_func(x):
  try:
    if x.pos_=="NOUN" or x.pos_=="VERB":
      return adapted_lesk(sent,x.text).name()
    else:
      return x.text
  except:
    return x.text

def wsd_text(sent):
  doc = nlp(sent)
  tmp_list = [one_func(i) for i in doc]
  return " ".join(tmp_list).strip()


wsd_text(sent)

'he is sit_down.v.01 on the bank.n.01 of the river.n.01 watch.v.06 a speedboat.n.01 .'

In [8]:
wn.synset('boat.n.01').hypernyms()
for i in wn.synset('boat.n.01').hypernym_paths()[0]:
  print(i.name())

entity.n.01
physical_entity.n.01
object.n.01
whole.n.02
artifact.n.01
instrumentality.n.03
conveyance.n.03
vehicle.n.01
craft.n.02
vessel.n.02
boat.n.01


In [9]:
df["WSD Text"] = df["Document"].map(wsd_text)

In [10]:
df.head()

,Document,Summary,WSD Text
0,japan 's nec corp. and UNK computer corp. of t...,nec UNK in computer sales tie-up,japan 's nec corp . and UNK computer corp . of...
1,the sri lankan government on wednesday announc...,sri lanka closes schools as war escalates,the sri lankan politics.n.02 on wednesday anno...
2,police arrested five anti-nuclear protesters t...,protesters target french research ship,police.n.01 halt.v.01 five anti - nuclear diss...
3,factory orders for manufactured goods rose #.#...,us september factory orders up #.# percent,factory.n.01 regulate.v.02 for manufacture.v.0...
4,the bank of japan appealed to financial market...,bank of UNK UNK for calm in financial markets,the bank of japan solicitation.n.02 to financi...


In [21]:
print(wn.synset("suppose.v.01").hypernym_paths()[0])
print(wn.synset("suppose.v.01").hypernyms())

[Synset('think.v.03'), Synset('reason.v.03'), Synset('speculate.v.02'), Synset('suppose.v.01')]
[Synset('speculate.v.02')]


In [16]:
a = df['WSD Text'].str.cat(sep=' ')
test_list = re.findall("[\w]+\..{1}\..{2}",a)
word_dist = FreqDist(test_list)


1

In [26]:
thetha_d = 3
thetha_f = 100

def to_generalize(sent,thetha_d,thetha_f,word_dist):
  word_lst = [word for word in re.findall('\w]+\..{1}\..{2}', sent) \
              if (word_dist[word] < thetha_f) and (len(wn.synset(word).hypernym_paths()[0]) > thetha_d)]
  
  for i in word_lst:
    sent = sent.replace(i, wn.synset(i).hypernyms())
  return sent

In [28]:
df["Generalized text"] = df["WSD Text"].map(lambda x: to_generalize(x,thetha_d,thetha_f,word_dist))
df.head()

,Document,Summary,WSD Text,Generalized text
0,japan 's nec corp. and UNK computer corp. of t...,nec UNK in computer sales tie-up,japan 's nec corp . and UNK computer corp . of...,japan 's nec corp . and UNK computer corp . of...
1,the sri lankan government on wednesday announc...,sri lanka closes schools as war escalates,the sri lankan politics.n.02 on wednesday anno...,the sri lankan politics.n.02 on wednesday anno...
2,police arrested five anti-nuclear protesters t...,protesters target french research ship,police.n.01 halt.v.01 five anti - nuclear diss...,police.n.01 halt.v.01 five anti - nuclear diss...
3,factory orders for manufactured goods rose #.#...,us september factory orders up #.# percent,factory.n.01 regulate.v.02 for manufacture.v.0...,factory.n.01 regulate.v.02 for manufacture.v.0...
4,the bank of japan appealed to financial market...,bank of UNK UNK for calm in financial markets,the bank of japan solicitation.n.02 to financi...,the bank of japan solicitation.n.02 to financi...
